In [1]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/*
import ROOT as root
from array import array
import math as math
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
is_sigmalised = 0
sim = 0
first_run = 1
extand_fit_range = 0
Min_Entries = 4000
N_files = 1
Do_save = 1

In [3]:
isigma = 2
N_layers = 8
layer_bins =[0,1,4,5,8,9,12,13]
N_hists = 4
layers = ["1-2 layer east","1-2 layer west","2-3 layer east","2-3 layer west",\
          "2-4 layer east","2-4 layer west","3-4 layer east","3-4 layer west",
          "3 layer east pos","3 layer west pos","3 layer east neg","3 layer west neg",\
          "4 layer east pos","4 layer west pos","4 layer east neg","4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/"
file_names="my-m_ee_Run14AuAu_96th_new_19915_809runs" 
#m_ee_Run14AuAu_95th_new_19914_1055runs m_ee_Run14AuAu_96th_new_19915_809runs
#m_ee_Run14AuAu_92nd_new_19900_1065runs m_ee_Run14AuAu_93rd_new_19904_100runs m_ee_Run14AuAu_94th_new_19913_317runs
#m_ee_Run14AuAu_84th_new_19850_778runs m_ee_Run14AuAu_83rd_new_19842_1065runs m_ee_Run14AuAu_85th_new_19858_1065runs m_ee_Run14AuAu_87th_new_19865_518runs
hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","theta"]

In [4]:
if sim :
    file_path="../sim/output/Newembed/dca/" 
    file_names="piminus_50M_new_v8"#piminus_50M_v7 piminus_50M_v8 piminus_50M_v9

In [5]:
hists_read = []

infile = root.TFile.Open(file_path+file_names+f".root", "read")
for ilayer in layer_bins:
    hist_select_3D0 = []
    for ihist in range(len(hist_select_3D_names)):
        hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
        hist_select_3D0[-1].SetDirectory(root.nullptr)
        hist_select_3D0[-1].Add(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer+2}"))
    hists_read.append(hist_select_3D0)

infile.Close()

In [6]:
delta_z = 3
z_bins = [i for i in range(8,30,delta_z)]
z_means = array('d', [i-20+delta_z/2 for i in z_bins])
z_error = array('d', [0.5*delta_z for i in z_bins])
z_bin_edges = [i for i in range(8,33,delta_z)]
print(z_bins,z_means, z_error,z_bin_edges)
pad_limits = [[1.8,4.4],[-1.2,1.3]]
N_phi = 6
phi_fit_range = [[[1.9,2.2],[2.2,2.6],[2.6,2.9],[2.9,3.3],[3.3,3.7],[3.7,4.3]],\
                 [[-1.2,-0.7],[-0.7,-0.2],[-0.2,0.0],[0.0,0.3],[0.3,0.7],[0.7,1.2]]]#26 33 37
keff=[0.001,0.001,0.0005,0.0005,0.0005,0.0005,0.00025,0.00025]
keff = [(9-6*is_sigmalised)*keff[i] for i in range(len(keff))]
if sim: keff = [keff[i]/100 for i in range(len(keff))]
R_layers = [2.64,5.22,5.22,11.7,16.6]
if extand_fit_range:
    phi_fit_range = [[[phi_fit_range[0][i][0], phi_fit_range[0][i][1]+0.1] for i in range(len(phi_fit_range[0]))],\
                     [[phi_fit_range[1][i][0], phi_fit_range[1][i][1]+0.1] for i in range(len(phi_fit_range[1]))]]
print(phi_fit_range)

[8, 11, 14, 17, 20, 23, 26, 29] array('d', [-10.5, -7.5, -4.5, -1.5, 1.5, 4.5, 7.5, 10.5]) array('d', [1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5]) [8, 11, 14, 17, 20, 23, 26, 29, 32]
[[[1.9, 2.2], [2.2, 2.6], [2.6, 2.9], [2.9, 3.3], [3.3, 3.7], [3.7, 4.3]], [[-1.2, -0.7], [-0.7, -0.2], [-0.2, 0.0], [0.0, 0.3], [0.3, 0.7], [0.7, 1.2]]]


In [ ]:
def sdphi_fitter(ihits, sigma=isigma):

    iHist = hists_read[ihits][iOption0+is_sigmalised*2]

    Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, phi_local, phi_local_err, int_fg = [], [], [], [], [], [], []

    for iz in range(len(z_bin_edges)-1):

        phi_z =  array( 'd' )
        phi_z_err =  array( 'd' )
        Mean_z =  array( 'd' )
        Sigma_z =  array( 'd' )
        Mean_z_err =  array( 'd' )
        Sigma_z_err =  array( 'd' )
        int_fg_local =  array( 'd' )

        N_ybins_loc = iHist.GetNbinsY()
        for iy in range(N_ybins_loc):
            proj_local = iHist.ProjectionX(iHist.GetName()+f"_{iz}_{iy}", iy+1, iy+1, z_bin_edges[iz], z_bin_edges[iz+1]-1)

            phi = iHist.GetYaxis().GetBinCenter(iy+1)
            phi_z.append(phi)
            phi_z_err.append(iHist.GetYaxis().GetBinWidth(iy+1)/2.)

            c1 = root.TCanvas("c1"+proj_local.GetName(),"c1"+proj_local.GetName(),720,360)
            gaus0 = root.TF1("gaus0"+proj_local.GetName(), "gaus", -0.025, 0.025)
            proj_local.Draw()
            proj_local.Fit(gaus0, "Q", "", -0.025, 0.025)
            gaus1 = root.TF1("gaus1"+proj_local.GetName(), "gaus", gaus0.GetParameter(1)-0.002,gaus0.GetParameter(1)+0.002)
            proj_local.Fit(gaus1, "Q", "", gaus0.GetParameter(1)-0.006,gaus0.GetParameter(1)+0.006)            
            gaus = root.TF1("gaus"+proj_local.GetName(), "gaus", gaus1.GetParameter(1)-gaus1.GetParameter(2)*2,gaus1.GetParameter(1)+gaus1.GetParameter(2)*2)
            proj_local.Fit(gaus, "Q", "", gaus1.GetParameter(1)-gaus1.GetParameter(2)*1.49,gaus1.GetParameter(1)+gaus1.GetParameter(2)*1.49)



            if proj_local.GetEntries() < Min_Entries or abs(gaus.GetParameter(1))>0.006:
                Mean_z.append(0)
                Sigma_z.append(0.002)
                Mean_z_err.append(0.003)
                Sigma_z_err.append(0.003)
                int_fg_local.append(1)
                continue
            
            c1.Update()
            if Do_save: c1.SaveAs("output/dynamic_delta_fits/" + proj_local.GetName() + ".png")
            Mean_z.append(gaus.GetParameter(1))
            Sigma_z.append(gaus.GetParameter(2))
            #Mean_z_err.append(min(gaus.GetParError(1)*1000, gaus.GetParameter(2)/3))
            #Sigma_z_err.append(min(gaus.GetParError(2), 0.05*gaus.GetParameter(2)))


            int_fg_local.append(gaus.Integral(gaus.GetParameter(1)-0.002,gaus.GetParameter(1)+0.002)+0.0001)
            #int_bg_local.append(2*gaus.Integral(gaus.GetParameter(1)+sigma*gaus.GetParameter(2),0.0025))
            Mean_z_err.append(keff[ihist]/abs(int_fg_local[-1])**0.5+0.00015)
            Sigma_z_err.append(keff[ihist]/abs(int_fg_local[-1])**0.5+0.00015)

            #m2_pos.Draw("same") m2_neg.Draw("same")
            #c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )
        Mean_local.append(Mean_z)
        Mean_local_err.append(Mean_z_err)
        Sigma_local.append(Sigma_z)
        Sigma_local_err.append(Sigma_z_err)
        phi_local.append(phi_z)
        phi_local_err.append(phi_z_err)
        int_fg.append(int_fg_local)

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, phi_local, phi_local_err, int_fg]


In [8]:
Ntr = N_layers
pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(sdphi_fitter, [i for i in range(Ntr)])
pool.close()

In [9]:
print(len(output_array[0][0]))

8


In [10]:
Means, Means_err, Sigmas, Sigmas_err, phis, phis_err, int_fg = [], [], [], [], [], [], []
for i in range(N_layers):
    Means.append(output_array[i][0])
    Means_err.append(output_array[i][1])
    Sigmas.append(output_array[i][2])
    Sigmas_err.append(output_array[i][3])
    phis.append(output_array[i][4])
    phis_err.append(output_array[i][5])
    int_fg.append(output_array[i][6])
    
#print("Means", len(Means), len(Means[0]), len(Means[0][1]))

In [11]:
g_sigma = []
h2=[]
c3 = root.TCanvas(f"c3",f"c3",1400,125*N_layers)
c3.Divide(2,int(N_layers/2),0,0)
Min, Max = 0., 0.003
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for ilayer in range(N_layers):
    c3.cd(ilayer+1)
    g_sigma_layer = []
    h2.append(Format_Pad_old(pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], Min, Max, "#phi [rad]", "#sigma#"+iOptames[iOption0] + " [mrad]", 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for ized in range(len(Sigmas[ilayer])):
        n_phi = len(Sigmas[ilayer][ized])
        if n_phi<1: continue
        g_sigma_layer.append(root.TGraphErrors(n_phi, phis[ilayer][ized], Sigmas[ilayer][ized],phis_err[ilayer][ized], Sigmas_err[ilayer][ized]))
        Format_Graph(g_sigma_layer[-1],20,1, ized, 3, ized,1,1)
        g_sigma_layer[-1].Draw("same p")
    g_sigma.append(g_sigma_layer)
    legends[-1].Draw()
        
    
    #g_sigma_f.append(g_sigma_layer)
#g_sigma.append(g_sigma_f)
c3.Draw()
#c3.SaveAs(f"output/note/sigma_{ilayer}{iOption0}{is_sigmalised}.pdf")

In [12]:
fit_mean_functions = []
for ilayer in range(N_layers):
    fit_functions = []
    for ized in range(len(z_bins)):
        fit_zed = []
        for iphi in range(N_phi):
            if not first_run:
                fit_zed.append(root.TF1(f"fit_functions_mean_iter_{ized}_"+layers[ilayer]+f"_{iphi}","[0]/10000*sin(x)+[1]/10000*cos(x)+[2]/10000",phi_fit_range[iOption0][iphi][0],phi_fit_range[iOption0][iphi][1]))
                fit_zed[-1].SetParameters(0.0, 0.0, 0.0)
            else:
                fit_zed.append(root.TF1(f"fit_functions_mean_iter_{ized}_"+layers[ilayer]+f"_{iphi}","[0]/10000*sin(x + 0.3/[3]*sin(x) + 0.04/[3]*cos(x))+[1]/10000*cos(x + 0.3/[3]*sin(x) + 0.04/[3]*cos(x))+[2]/10000",phi_fit_range[iOption0][iphi][0],phi_fit_range[iOption0][iphi][1]))
                fit_zed[-1].SetParameters(0.0, 0.0, 0.0, 0.0)
                fit_zed[-1].FixParameter(3,R_layers[int(ilayer/2)])

            fit_zed[-1].FixParameter(2,0.00)
            fit_zed[-1].SetParLimits(0, -25, 25)#25,75
            fit_zed[-1].SetParLimits(1, -25, 25)#25,75
            #fit_functions[-1].FixParameter(1,0.00)
            fit_zed[-1].SetLineColor(ized+1)
        fit_functions.append(fit_zed)
    fit_mean_functions.append(fit_functions)

In [ ]:
g_mean = []
h2=[]
c3 = root.TCanvas(f"c3",f"c3",1400,125*N_layers)
c3.Divide(2,int(N_layers/2),0,0)
Min, Max = -0.005, 0.005
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for ilayer in range(N_layers):
    arm = ilayer%2
    c3.cd(ilayer+1)
    g_mean_layer = []
    h2.append(Format_Pad_old( pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], Min, Max, "#phi [rad]", "#LT#Delta#"+iOptames[iOption0]+"#GT [mrad]", 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    legends.append(root.TLegend(0.2,0.8,0.45,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for ized in range(len(Means[ilayer])):
        n_phi = len(Means[ilayer][ized])
        if n_phi<1: continue
        g_sigma_layer.append(root.TGraphErrors(n_phi, phis[ilayer][ized], Means[ilayer][ized],phis_err[ilayer][ized], Means_err[ilayer][ized]))
        Format_Graph(g_sigma_layer[-1],20,1, ized+1, 3, ized+1,1,1)
        g_sigma_layer[-1].Draw("same p")
        for iphi in range(N_phi):
            g_sigma_layer[-1].Fit(fit_mean_functions[ilayer][ized][iphi],"QR+", "", phi_fit_range[arm][iphi][0],phi_fit_range[arm][iphi][1])
    g_sigma.append(g_sigma_layer)
    legends[-1].Draw()
        
    
    #g_sigma_f.append(g_sigma_layer)
#g_sigma.append(g_sigma_f)
c3.Draw()
c3.SaveAs(f"output/real_mean_corr.png")

In [14]:
parameters = []
par_errors = []
for ilayer in range(N_layers):
    parameters_layer, par_errors_layer = [], []
    for ized in range(len(z_bins)):
        parameters_zed, par_errors_zed = [], []
        for iphi in range(N_phi):
            parameters_phi, par_errors_phi = array('d'), array('d')
            for ipar in range(3):
                if ipar==0:
                    parameters_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParameter(ipar))
                    par_errors_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParError(ipar))
                if ipar==1:
                    parameters_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParameter(ipar))
                    par_errors_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParError(ipar))
                if ipar==2:
                    parameters_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParameter(ipar))
                    par_errors_phi.append(fit_mean_functions[ilayer][ized][iphi].GetParError(ipar))
            parameters_zed.append(parameters_phi)
            par_errors_zed.append(par_errors_phi)
        parameters_layer.append(parameters_zed)
        par_errors_layer.append(par_errors_zed)
    parameters.append(parameters_layer)
    par_errors.append(par_errors_layer)

In [15]:
c2 = root.TCanvas(f"c2",f"c2",1400,125*N_layers)
c2.Divide(2,int(N_layers/2),0,0)
graphs_par = []
h2 = []
legends = []
for ilayer in range(N_layers):
    c2.cd(ilayer+1)
    h2.append(Format_Pad_old(-11,11, -50, 50, "z [cm]", "Parameter value [mum]", 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    n_zed = len(z_bins)
    legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0) 
    for ipar in range(0,2):
        # Collect parameter values and errors for this layer and parameter
        for iphi in range(N_phi):
            par_vals = [parameters[ilayer][ized][iphi][ipar] for ized in range(n_zed)]
            par_errs = [par_errors[ilayer][ized][iphi][ipar] for ized in range(n_zed)]
            graphs_par.append(root.TGraphErrors(n_zed, z_means, array('d', par_vals), z_error, array('d', par_errs)))
            graphs_par[-1].SetMarkerStyle(20 + ipar%3)
            graphs_par[-1].SetMarkerColor(1 + iphi)
            graphs_par[-1].SetLineColor(1 + iphi)
            if iphi == 0:
                graphs_par[-1].SetTitle(f"{layers[ilayer]}: Parameter {ipar} vs z")
                graphs_par[-1].Draw("same p")
            else:
                graphs_par[-1].Draw("P SAME")
    legends[ilayer].AddEntry(graphs_par[-3], "Parameter 0", "p")
    legends[ilayer].AddEntry(graphs_par[-2], "Parameter 1", "p")
    #legends[ilayer].AddEntry(graphs_par[-1], "Parameter 2", "p")    
    legends[ilayer].Draw()
c2.Draw()
c2.SaveAs(f"output/real_parameters.png")

In [16]:
# Print parameters as a C++ array initialization
if True:
    print(f"const float VTX_seg_alignment[{len(parameters)-2}][{n_zed}][{N_phi}][3] = {{")
    for ilayer in range(len(parameters)):
        if ilayer ==4 or ilayer == 5:  # Skip layers 4 and 5
            continue
        print("  {")
        for ized in range(n_zed):
            print("    {")
            for iphi in range(N_phi):
                vals = parameters[ilayer][ized][iphi]
                print(f"      {{{vals[0]:.4g}, {vals[1]:.4g}, {vals[2]:.4g}}}", end="")
                if iphi != N_phi - 1:
                    print(",", end="")
                print()
            print("    }" + ("," if ized != n_zed - 1 else ""))
        print("  }" + ("," if ilayer != len(parameters) - 1 else ""))
    print("};")

const float VTX_seg_alignment[6][8][6][3] = {
  {
    {
      {-2.138, 4.888, 0},
      {15.91, 11.22, 0},
      {-12.24, -3.311, 0},
      {11.36, 1.774, 0},
      {-13.18, 3.228, 0},
      {3.572, 1.117, 0}
    },
    {
      {3.763, 5.904, 0},
      {21.31, 12.37, 0},
      {-2.894, 0.3828, 0},
      {3.623, -0.2621, 0},
      {-3.816, -0.4116, 0},
      {3.641, -3.16, 0}
    },
    {
      {9.828, 8.977, 0},
      {12.14, 4.061, 0},
      {-1.725, -0.7903, 0},
      {3.739, -0.8856, 0},
      {-6.169, 0.8924, 0},
      {0.4359, 0.06125, 0}
    },
    {
      {8.34, 5.972, 0},
      {8.662, 1.424, 0},
      {12.56, 5.041, 0},
      {9.552, -1.155, 0},
      {-6.864, 0.5506, 0},
      {0.7511, -0.871, 0}
    },
    {
      {-0.5582, -2.337, 0},
      {9.764, 3.337, 0},
      {23.25, 8.603, 0},
      {7.912, -0.9623, 0},
      {-3.448, 0.07383, 0},
      {1.782, -0.7034, 0}
    },
    {
      {8.052, 12.25, 0},
      {4.149, 0.5214, 0},
      {24.41, 11.38, 0},
      {14.51, -0.5219, 

In [17]:
# Example for plotting means in 3D for ilayer=0
ilayer = 0
g3d = root.TGraph2D()
point = 0
print(len(z_bins), len(Means[ilayer]), len(Means[ilayer][0]))
for ized, z in enumerate(z_bins):
    n_phi = len(Means[ilayer][ized])
    for iphi in range(n_phi):
        phi = phis[ilayer][ized][iphi]
        mean = Means[ilayer][ized][iphi]
        g3d.SetPoint(point, phi, z_means[ized], mean)
        point += 1
# Define the fit function: ([0]+[1]*zed)*sin(phi)+([2]+[3]*zed)*cos(phi)+[4]
fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x)+([2]+[3]*y)/10000*cos(x)+[4]/1000", -2, 4, min(z_means), max(z_means))
fit_func.SetParameters(0, 0, 0, 0, 0)  # You can tune these

c3d = root.TCanvas("c3d", "3D Mean", 800, 600)


g3d.SetTitle("Mean vs #phi and z;#phi;z;Mean")
g3d.SetMarkerStyle(20)
g3d.SetMarkerColor(root.kBlue)
# Fit the 3D graph
g3d.Fit(fit_func, "Q")  # "Q" for quiet

# Draw the points and the fit surface
g3d.Draw("P0 surf1")
fit_func.Draw("same surf1")

#c3d.Draw()

8 8 60


In [18]:
# Print the fit parameters and their errors for the 3D fit function
for i in range(5):
    par = fit_func.GetParameter(i)
    err = fit_func.GetParError(i)
    print(f"Parameter [{i}]: {par:.2g} ± {err:.2g}")

Parameter [0]: 0.34 ± 0.11
Parameter [1]: 0.0033 ± 0.016
Parameter [2]: -0.33 ± 0.1
Parameter [3]: -0.017 ± 0.015
Parameter [4]: 0.021 ± 0.0076


In [19]:
# Fill a TH3D with (phi, z, mean) and fit 2D slices with the given function
layers3D = [0]
ilayer = layers3D[0]  # Example for the first layer in 3D
# Define binning
n_bins_phi = 20
n_bins_z = len(z_means)
phi_min, phi_max = pad_limits[ilayer%2][0], pad_limits[ilayer%2][1]
z_min, z_max = min(z_means), max(z_means)
mean_min, mean_max = -0.002, 0.002

th3 = root.TH3D("th3", "Mean vs #phi and z;#phi [rad];z [cm];#LT#Delta#phi#GT [mrad]", n_bins_phi, pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], int(n_bins_z), z_min, 0, 100, mean_min, mean_max)
# Fill TH3D
for ilayer in layers3D:
    for ized, z in enumerate(z_bins):
        n_phi = len(Means[ilayer][ized])
        for iphi in range(n_phi):
            phi = phis[ilayer][ized][iphi]
            mean = Means[ilayer][ized][iphi]
            mean_err = int_fg[ilayer][ized][iphi]**0.5
            th3.Fill(phi, z_means[ized], mean, int_fg[ilayer][ized][iphi])
            #print(th3.FindBin(phi, z_means[ized], mean))
            ibin = th3.FindBin(phi, z_means[ized], mean)
            th3.SetBinError(ibin, th3.GetBinContent(ibin)**0.5)  # Set the error as the square root of the content

c1 = root.TCanvas("c1", "3D Histogram", 800, 600)
c1.SetLeftMargin(0.)
th3.Draw("LEGO2")  # Draw the 3D histogram
if False:#for first stupid run
    fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x + 0.3/[5]*sin(x) + 0.04/[5]*cos(x))+([2]+[3]*y)/10000*cos(x + 0.3/[5]*sin(x) + 0.04/[5]*cos(x))+[4]/1000", phi_min, phi_max, min(z_means), max(z_means))
    fit_func.FixParameter(5, R_layers[int(ilayer/2)])  # Fix the constant term to zero for the fit
else:
    fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x)+([2]+[3]*y)/10000*cos(x)+[4]/1000", phi_min, phi_max, min(z_means), max(z_means))
    fit_func.SetParameters(0, 0, 0, 0, 0)  # You can tune these
fit_func.FixParameter(4, 0)  # Fix the constant term to zero for the fit
th3.Fit(fit_func, "Q")  # Fit the 3D histogram
fit_func.Draw("same surf1")
c1.Draw()

# fit_results now contains the fit parameters for each z slice

In [20]:
print(f"{-fit_func.GetParameter(0):.3g} , {-fit_func.GetParameter(1):.3g}")
print(f"{-fit_func.GetParameter(2):.3g} , {-fit_func.GetParameter(3):.3g}")

-2.32 , -0.23
2.08 , 0.207


In [21]:
#print(f"chi2/ndf = {fit_func.GetChisquare():.2f} / {fit_func.GetNDF()} = {fit_func.GetChisquare()/fit_func.GetNDF():.2f}")
# Print the fit parameters and their errors for the 3D fit function
for i in range(5):
    par = fit_func.GetParameter(i)
    err = fit_func.GetParError(i)
    print(f"Parameter [{i}]: {par:.2g} ± {err:.2g}")

Parameter [0]: 2.3 ± 0.73
Parameter [1]: 0.23 ± 0.16
Parameter [2]: -2.1 ± 0.35
Parameter [3]: -0.21 ± 0.077
Parameter [4]: 0 ± 0


In [22]:
#print(fit_func.Eval(3.14, 0))  # Example evaluation at phi=3.0, z=20
# Print the equation for dphi(z, phi) with obtained fit parameters
print(
    f"dphi = ({fit_func.GetParameter(0):.6g} + {fit_func.GetParameter(1):.6g}*z)/10000 * sin(phi) + "
    f"({fit_func.GetParameter(2):.6g} + {fit_func.GetParameter(3):.6g}*z)/10000 * cos(phi) + "
    f"{fit_func.GetParameter(4)/1000:.6g}"
)
print(f"x + ({fit_func.GetParameter(0):.3g} + {fit_func.GetParameter(1):.3g}*z)/10000")
print(f"y + ({fit_func.GetParameter(2):.3g} + {fit_func.GetParameter(3):.3g}*z)/10000")

dphi = (2.32144 + 0.229839*z)/10000 * sin(phi) + (-2.08208 + -0.207166*z)/10000 * cos(phi) + 0
x + (2.32 + 0.23*z)/10000
y + (-2.08 + -0.207*z)/10000


In [23]:
print(f"x + ({fit_func.GetParameter(0):.3g} + {fit_func.GetParameter(1):.3g}*z)/10000 + r * sin( {fit_func.GetParameter(4):.3g}/1000)*sin(phi)")
print(f"y + ({fit_func.GetParameter(2):.3g} + {fit_func.GetParameter(3):.3g}*z)/10000 + r * sin( {fit_func.GetParameter(4):.3g}/1000)*cos(phi)")

x + (2.32 + 0.23*z)/10000 + r * sin( 0/1000)*sin(phi)
y + (-2.08 + -0.207*z)/10000 + r * sin( 0/1000)*cos(phi)


## Formula for Alignment Fit

The function used for fitting the mean shift as a function of azimuthal angle (φ) and longitudinal position (z) is:

<div align="center">

Δφ(z, φ) = ([0] + [1]·z)/10000 · sin(φ) + ([2] + [3]·z)/10000 · cos(φ) + [4]/1000

</div>

**where:**
- [0], [1], [2], [3], [4] are fit parameters extracted from the data,
- z is the longitudinal coordinate (e.g., along the beam axis),
- φ is the azimuthal angle.

---

### Why use this function for a 3D histogram fit?

- **Physical Motivation:**  
  Misalignments of detector layers often manifest as sinusoidal modulations in the measured hit positions as a function of φ, with possible linear dependence on z due to tilts or twists.

- **Parameter Extraction:**  
  Fitting this function to the 3D histogram of mean residuals (mean shift vs. φ and z) allows extraction of alignment parameters:
    - The coefficients of sin(φ) and cos(φ) (and their z-dependence) correspond to translations and tilts of the detector layer in the transverse plane.
    - The constant term can represent a global offset.

- **Usage:**  
  After fitting, the obtained parameters can be used to correct (align) the detector geometry, improving tracking and vertexing performance.

In [24]:
first_ieter = [
    [
        [
            [3.85, 9.305, 0],
            [5.881, 1.616, 0],
            [34.79, 18.37, 0],
            [-32.94, 11.65, 0],
            [27.38, 4.751, 0],
            [-3.246, 7.63, 0]
        ],
        [
            [18.4, 31.37, 0],
            [10.29, 4.967, 0],
            [20.4, 12.12, 0],
            [10.91, 11.02, 0],
            [29.35, 7.201, 0],
            [-10.76, 29.19, 0]
        ],
        [
            [19.55, 30.38, 0],
            [1.568, -3.193, 0],
            [19.87, 7.078, 0],
            [21.33, 7.366, 0],
            [22.07, 3.054, 0],
            [-13.69, 23.45, 0]
        ],
        [
            [19.02, 26.93, 0],
            [6.019, -1.313, 0],
            [6.316, 1.33, 0],
            [24.42, 3.423, 0],
            [21.12, -2.029, 0],
            [-21.08, 21.14, 0]
        ],
        [
            [10.46, 20.31, 0],
            [-5.533, -8.68, 0],
            [4.135, -1.625, 0],
            [19.45, -0.4266, 0],
            [16.2, -5.819, 0],
            [-16.72, 10.53, 0]
        ],
        [
            [14.32, 31.84, 0],
            [-28.74, -19.12, 0],
            [-9.132, -4.39, 0],
            [10.55, -3.96, 0],
            [4.602, -5.711, 0],
            [-20.29, 4.149, 0]
        ],
        [
            [14.61, 34.13, 0],
            [-21.54, -10.31, 0],
            [-28.83, -8.179, 0],
            [1.833, -6.281, 0],
            [15.64, -11.23, 0],
            [-23.94, 6.761, 0]
        ],
        [
            [15.01, 39.26, 0],
            [-32.02, -14.36, 0],
            [-39.11, -9.984, 0],
            [-12.18, -9.14, 0],
            [7.466, -14.96, 0],
            [-23.12, -3.734, 0]
        ]
    ],
    [
        [
            [0, 0, 0],
            [-18.01, -9.912, 0],
            [-11.36, -9.503, 0],
            [30.01, -14.88, 0],
            [-12.63, 4.7, 0],
            [0, 0, 0]
        ],
        [
            [-5.278, -16.36, 0],
            [-4.601, -6.267, 0],
            [-16.8, -8.136, 0],
            [36.43, -15.32, 0],
            [-13.55, 8.152, 0],
            [16.19, -13.95, 0]
        ],
        [
            [-10.52, -22.91, 0],
            [-5.515, -9.783, 0],
            [4.519, -8.722, 0],
            [20.08, -12.6, 0],
            [-2.648, -1.152, 0],
            [6.733, -3.528, 0]
        ],
        [
            [-15.99, -27.78, 0],
            [-10.06, -11.31, 0],
            [75, -0.7069, 0],
            [-21.69, 5.978, 0],
            [-3.956, 1.365, 0],
            [4.971, -4.392, 0]
        ],
        [
            [-15.88, -22.53, 0],
            [-8.743, -7.475, 0],
            [75, 3.66, 0],
            [-47.43, 18.22, 0],
            [3.409, -0.9332, 0],
            [6.11, -7.666, 0]
        ],
        [
            [-20.88, -28.87, 0],
            [4.452, -1.419, 0],
            [49.1, 3.003, 0],
            [-5.66, 7.025, 0],
            [4.565, 4.319, 0],
            [1.08, 2.818, 0]
        ],
        [
            [-11.9, -22.44, 0],
            [10.75, 3.358, 0],
            [36.23, 5.944, 0],
            [-15.36, 9.762, 0],
            [26.23, -7.065, 0],
            [-6.729, 15.26, 0]
        ],
        [
            [-9.811, -24.9, 0],
            [23.4, 8.64, 0],
            [32.3, 9.873, 0],
            [-28.43, 15.11, 0],
            [32.62, -10.16, 0],
            [-10.66, 23.09, 0]
        ]
    ],
    [
        [
            [-18.58, -38.69, 0],
            [17.03, 4.495, 0],
            [-35.69, -18.2, 0],
            [56.81, -2.07, 0],
            [-23.91, 6.818, 0],
            [0, 0, 0]
        ],
        [
            [-4.646, -13.22, 0],
            [31.73, 15.22, 0],
            [-35.28, -16.56, 0],
            [36.02, -3.221, 0],
            [-31.74, 7.97, 0],
            [19.9, -24.22, 0]
        ],
        [
            [3.731, 5.705, 0],
            [16.71, 9.281, 0],
            [-23.69, -9.418, 0],
            [16.85, -0.6678, 0],
            [-21.07, 6.379, 0],
            [14.66, -13.38, 0]
        ],
        [
            [12.34, 25.67, 0],
            [20.11, 13.56, 0],
            [-25.34, -8.32, 0],
            [15.94, 1.759, 0],
            [-33.18, 10.11, 0],
            [5.249, -13.59, 0]
        ],
        [
            [15.01, 35.06, 0],
            [14.69, 14.07, 0],
            [-19.16, -2.411, 0],
            [18.69, 6.945, 0],
            [-22.39, 7.767, 0],
            [-12.34, 1.548, 0]
        ],
        [
            [19.88, 49.81, 0],
            [-7.242, 2.953, 0],
            [-20.79, -1.935, 0],
            [-6.74, 4.881, 0],
            [-21.2, 7.832, 0],
            [-20.06, 7.886, 0]
        ],
        [
            [15.56, 51.58, 0],
            [-5.275, 10.1, 0],
            [-21.99, 2.343, 0],
            [-6.652, 8.4, 0],
            [-29.19, 13.46, 0],
            [-9.64, 1.553, 0]
        ],
        [
            [16.79, 57.68, 0],
            [-22.88, 3.441, 0],
            [-23.06, 4.113, 0],
            [-11.73, 8.201, 0],
            [-26.37, 10.65, 0],
            [-12.82, 1.43, 0]
        ]
    ],
    [
        [
            [-13.29, -23.67, 0],
            [27.41, 9.577, 0],
            [69.26, 13.66, 0],
            [-22.67, 20.05, 0],
            [39.08, -4.095, 0],
            [0, 0, 0]
        ],
        [
            [-25.05, -52.86, 0],
            [32.17, 5.783, 0],
            [70.79, 8.168, 0],
            [-14.26, 13.16, 0],
            [40.67, -8.27, 0],
            [37.95, -9.192, 0]
        ],
        [
            [-15.72, -39.12, 0],
            [27.59, 4.176, 0],
            [48.98, 4.687, 0],
            [-11.65, 7.97, 0],
            [26.71, -6.658, 0],
            [28.18, -8.889, 0]
        ],
        [
            [-12.76, -33.41, 0],
            [16.4, 0.5418, 0],
            [-29.62, -6.132, 0],
            [25.07, -11.38, 0],
            [15.04, -7.207, 0],
            [20.57, -11.72, 0]
        ],
        [
            [-5.717, -14.33, 0],
            [-4.148, -3.998, 0],
            [-61.77, -11.63, 0],
            [39.23, -20.28, 0],
            [5.7, -6.648, 0],
            [16.29, -16.51, 0]
        ],
        [
            [-16.59, -35.87, 0],
            [-1.367, -4.263, 0],
            [-5.442, -5.419, 0],
            [4.825, -9.902, 0],
            [-2.994, -4.883, 0],
            [6.993, -13.6, 0]
        ],
        [
            [-19.45, -31.2, 0],
            [1.841, -1.533, 0],
            [35.97, 1.347, 0],
            [-33.45, 5.144, 0],
            [-17.83, 1.009, 0],
            [0.7767, -14.04, 0]
        ],
        [
            [-22.1, -38.26, 0],
            [-4.816, -8.506, 0],
            [44.65, -2.415, 0],
            [-42.46, 3.368, 0],
            [-18.65, -0.9044, 0],
            [-3.026, -12.9, 0]
        ]
    ],
    [
        [
            [21.01, 47.25, 0],
            [-11.68, -1.554, 0],
            [0.3368, 4.234, 0],
            [-24.05, 5.243, 0],
            [10.13, 2.113, 0],
            [2.674, -0.5902, 0]
        ],
        [
            [0, 0, 0],
            [0.4843, 4.583, 0],
            [10.18, 8.942, 0],
            [-15.73, 6.221, 0],
            [45.36, -7.457, 0],
            [-11.85, 23.86, 0]
        ],
        [
            [15.63, 36.62, 0],
            [-18.31, -7.595, 0],
            [2.894, 3.476, 0],
            [-7.921, 2.487, 0],
            [11.68, -2.686, 0],
            [-20.15, 16.23, 0]
        ],
        [
            [15.03, 32.94, 0],
            [-9.716, -2.731, 0],
            [0.6464, 2.583, 0],
            [-10.11, 1.45, 0],
            [9.289, -2.283, 0],
            [-19.5, 16.98, 0]
        ],
        [
            [16.54, 40.39, 0],
            [-15.71, -6.519, 0],
            [1.124, 2.534, 0],
            [-9.899, 0.829, 0],
            [1.952, -2.585, 0],
            [-6.084, 5.921, 0]
        ],
        [
            [12.16, 30.66, 0],
            [-5.993, -0.6234, 0],
            [-10.88, -2.775, 0],
            [9.898, 1.11, 0],
            [-1.919, -2.428, 0],
            [2.312, -1.406, 0]
        ],
        [
            [10.69, 25.04, 0],
            [2.883, 4.165, 0],
            [-14.6, -4.946, 0],
            [5.605, -1.064, 0],
            [-0.8109, -2.832, 0],
            [-4.547, -5.52, 0]
        ],
        [
            [8.014, 19.91, 0],
            [-5.718, -1.721, 0],
            [-16.77, -7.36, 0],
            [9.271, -2.239, 0],
            [3.153, -3.63, 0],
            [-13.45, -0.9366, 0]
        ]
    ],
    [
        [
            [-3.941, -9.348, 0],
            [5.818, -2.159, 0],
            [17.09, -0.01929, 0],
            [8.068, -0.2433, 0],
            [0.2248, 2.651, 0],
            [-2.093, 5.17, 0]
        ],
        [
            [1.468, -3.947, 0],
            [6.991, 2.314, 0],
            [7.597, 2.945, 0],
            [-3.791, 3.682, 0],
            [3.699, -0.01858, 0],
            [-4.054, 5.311, 0]
        ],
        [
            [0.07525, -13.12, 0],
            [11.42, 0.4315, 0],
            [3.008, -0.1124, 0],
            [3.564, -0.9782, 0],
            [2.975, -1.251, 0],
            [2.466, -2.272, 0]
        ],
        [
            [-8.228, -18.57, 0],
            [14.77, 2.887, 0],
            [-0.9126, 0.8864, 0],
            [3.465, -0.6491, 0],
            [2.071, -0.4637, 0],
            [-1.69, 1.144, 0]
        ],
        [
            [-1.064, -6.774, 0],
            [7.761, 0.7065, 0],
            [-7.655, -0.9884, 0],
            [8.08, -3.486, 0],
            [3.688, -2.049, 0],
            [1.233, -0.5603, 0]
        ],
        [
            [2.412, 0.4222, 0],
            [6.37, 1.187, 0],
            [-4.127, 0.6228, 0],
            [-2.581, -0.6086, 0],
            [4.062, -3.47, 0],
            [-3.44, 2.688, 0]
        ],
        [
            [2.79, -2.297, 0],
            [3.717, 0.7271, 0],
            [-12.41, -0.8844, 0],
            [-5.895, -1.531, 0],
            [7.226, -6.968, 0],
            [1.49, -4.619, 0]
        ],
        [
            [0, 0, 0],
            [0.8769, 0.3677, 0],
            [-16.74, -1.645, 0],
            [-6.048, -2.345, 0],
            [7.021, -7.764, 0],
            [4.946, -9.407, 0]
        ]
    ]
]


In [25]:
if True:
    print(f"const float VTX_seg_alignment[{len(parameters)-2}][{n_zed}][{N_phi}][3] = {{")
    jlayer = -1
    for ilayer in range(len(parameters)):
        if ilayer ==4 or ilayer == 5:  # Skip layers 4 and 5
            continue
        jlayer += 1
        print("  {")
        for ized in range(n_zed):
            print("    {")
            for iphi in range(N_phi):
                vals = parameters[ilayer][ized][iphi]
                print(f"      {{{vals[0]+first_ieter[jlayer][ized][iphi][0]:.4g}, {vals[1]+first_ieter[jlayer][ized][iphi][1]:.4g}, {vals[2]+first_ieter[jlayer][ized][iphi][2]:.4g}}}", end="")
                if iphi != N_phi - 1:
                    print(",", end="")
                print()
            print("    }" + ("," if ized != n_zed - 1 else ""))
        print("  }" + ("," if ilayer != len(parameters) - 1 else ""))
    print("};")

const float VTX_seg_alignment[6][8][6][3] = {
  {
    {
      {1.712, 14.19, 0},
      {21.79, 12.84, 0},
      {22.55, 15.06, 0},
      {-21.58, 13.42, 0},
      {14.2, 7.979, 0},
      {0.3264, 8.747, 0}
    },
    {
      {22.16, 37.27, 0},
      {31.6, 17.34, 0},
      {17.51, 12.5, 0},
      {14.53, 10.76, 0},
      {25.53, 6.789, 0},
      {-7.119, 26.03, 0}
    },
    {
      {29.38, 39.36, 0},
      {13.71, 0.8676, 0},
      {18.15, 6.288, 0},
      {25.07, 6.48, 0},
      {15.9, 3.946, 0},
      {-13.25, 23.51, 0}
    },
    {
      {27.36, 32.9, 0},
      {14.68, 0.1105, 0},
      {18.88, 6.371, 0},
      {33.97, 2.268, 0},
      {14.26, -1.478, 0},
      {-20.33, 20.27, 0}
    },
    {
      {9.902, 17.97, 0},
      {4.231, -5.343, 0},
      {27.39, 6.978, 0},
      {27.36, -1.389, 0},
      {12.75, -5.745, 0},
      {-14.94, 9.827, 0}
    },
    {
      {22.37, 44.09, 0},
      {-24.59, -18.6, 0},
      {15.27, 6.99, 0},
      {25.06, -4.482, 0},
      {3.349, -6.145, 0},
 

In [26]:
pt_fit = [0.25,   0.35, 0.45,   0.55,  0.75, 1.05,2.05, 3.05, 4.05]
pt_fit = array('d', pt_fit)
sigma = [0.005, 0.005, 0.004, 0.0035, 0.003, 0.002, 0.0019, 0.0018, 0.0017]
sigma = array('d', sigma)
c4 = root.TCanvas("c4", "Sigma vs pT", 800, 600)
g_sigma_pt = root.TGraph(len(pt_fit), pt_fit, sigma)
Format_Graph(g_sigma_pt, 20, 1, 1, 3, 1, 1, 1)
fit_func_res = root.TF1("fit_func_res", "2*sqrt([0]*[0]+[1]*[1]/x/x)", 0.2, 4.5)
fit_func_res.SetParameters(0.0012, 0.001)
#g_sigma_pt.Fit("", "QR+", "", 0.2, 4.5)
#g_sigma_pt.Fit(fit_func_res, "QR+", "", 0.4, 4.5)
g_sigma_pt.Draw("AP")
fit_func_res.Draw("same")
c4.Draw()